In [1]:
import twitter_data
import datetime as dt
import pandas as pd

In [2]:
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', 40)
pd.options.display.max_columns = 10000

In [4]:
start_date = dt.date(year=2021, month=2, day=1)
end_date = dt.date(year=2021, month=3, day=28)

In [5]:
test_df = twitter_data.generate_df(['GNUS', 'XSPA', 'IBIO', 'GME', 'OPES'], start_date, end_date)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


TweepError: Twitter error response: status code = 429

In [ ]:
test_df.index.min()

In [ ]:
test_df.index.max()

In [ ]:
test_df.head()

In [ ]:
print(test_df.shape)

Using the current approach (from project deliverable 2) results in either not a full date range or it maxes out the number of api calls. 

In [ ]:
test_df.to_parquet('2021-03-01_to_2021-03-23.parquet')

In [6]:
test_df.dtypes

NameError: name 'test_df' is not defined

In [ ]:
test_df[test_df['retweet_count'] > 100].iloc[0]

In [ ]:
test_df[test_df['user.followers_count'] > 10000].iloc[0]

In [ ]:
test_df[test_df['retweeted'] == False].iloc[0]

In [7]:
# Testing list generation
test_list = twitter_data.get_stock_tweets_list('GNUS', '2021-2-22')

TweepError: Twitter error response: status code = 429

In [ ]:
print(len(test_list))

In [ ]:
start_date = dt.date(year=2021, month=2, day=22)
end_date = dt.date(year=2021, month=3, day=23)
print(start_date)
print(end_date)

In [ ]:
# Test 
test_list_dt = twitter_data.get_stock_tweets_list('GNUS', start_date)

In [ ]:
print(len(test_list_dt))

In [ ]:
start_date = dt.date(year=2021, month=1, day=21)
end_date = dt.date(year=2021, month=3, day=22)
print(start_date)
print(end_date)

In [ ]:
PTTN_df = twitter_data.generate_df(['DLPN'], start_date, None)

In [ ]:
PTTN_df.index.min()

In [ ]:
PTTN_df.index.max()

In [ ]:
PTTN_df.head()

In [ ]:
PTTN_df.to_parquet('DLPN_2021_3_19_to_2021_3_24.parquet')

In [ ]:
PTTN_df = PTTN_df.reset_index()
PTTN_df['created_at'] = pd.to_datetime(PTTN_df['created_at'])

In [ ]:
PTTN_df = PTTN_df.set_index('created_at')

In [ ]:
tweets_bucketed = PTTN_df.groupby([pd.Grouper(freq='5min')]).count().reset_index()[['created_at','id']]
tweets_bucketed.head()

In [ ]:
tweets_bucketed_followers = PTTN_df[PTTN_df['user.followers_count'] > 1000].groupby([pd.Grouper(freq='5min')]).count().reset_index()[['created_at','id']]
tweets_bucketed_followers.head()

In [ ]:
all_tweets = pd.merge(tweets_bucketed, tweets_bucketed_followers, on='created_at')
all_tweets.rename(columns={'id_x': 'total_tweets', 'id_y': 'over_1k_followers'}, inplace=True)
all_tweets.head()

In [ ]:
all_tweets.to_parquet('DLPN_1k.parquet')

In [ ]:
start_date = dt.date(year=2020, month=6, day=1)
end_date = dt.date(year=2021, month=3, day=22)
print(start_date)
print(end_date)

In [ ]:
GNUS_df = twitter_data.generate_df(['GNUS'], start_date, end_date)

In [ ]:
GNUS_df.index.max()

In [ ]:
GNUS_df.index.min()

# Alternate Approaches

In [2]:
import os
import json
import tweepy

In [3]:
# cur_path = os.path.dirname(__file__)
file_path = 'keys.json'

with open(file_path) as keys:
    config = json.load(keys)
auth = tweepy.OAuthHandler(config['twitter']['API_key'], config['twitter']['API_secret_key'])

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
# checking the current keys
for key in config['twitter'].items():
    print(key)

('API_key', 'I0xHy3s5hn5PkUtgdUoI2n26j')
('API_secret_key', 'Sm6sZLZPdS9qBxk8jeLItFkG74oRykU2ngNxuEtZzwudzTLfUH')
('Bearer_token', '')
('Access_token', '1366053196502810624-44ymItMYIq7Ni2ZqHIpFKj7cSCpgQK')
('Access_token_secret', '8Gb5nRZ19YQinf3K1b2ESDx5bAQNMzXk23jLOnIsVsQZV')


In [6]:
print(auth.access_token)

None


In [7]:
print(auth.access_token_secret)

None


In [8]:
auth.set_access_token(config['twitter']['Access_token'], config['twitter']['Access_token_secret'])

In [9]:
print(auth.access_token)
print(auth.access_token_secret)

1366053196502810624-44ymItMYIq7Ni2ZqHIpFKj7cSCpgQK
8Gb5nRZ19YQinf3K1b2ESDx5bAQNMzXk23jLOnIsVsQZV


In [10]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [8]:
# Defining current list function 
def get_stock_tweets_list(stock, start_date, end_date=None):
#     search_tweets = []
#     c = tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date, include_entities=True).items()
#     while True:
#         try:
#             tweet = c.next()
#             # Insert into db
#             search_tweets.append(tweet)
#         except tweepy.TweepError:
#             print(tweet.created_at)
#             # time.sleep(60 * 15)
#             continue
#         except StopIteration:
#             break
    
    
    # statuses = tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date).items()
    
    # search_tweets.append()
    searched_tweets = [status for status in tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date).items()]
    return searched_tweets

In [9]:
def generate_df(stocks, start_date, end_date=None):
    data = pd.DataFrame()
    for stock in stocks:
        tweets = get_stock_tweets_list(stock, start_date, end_date)
        json_data = [tweet._json for tweet in tweets]
        stock_data = pd.io.json.json_normalize(json_data)
        stock_data['stock'] = stock
        data = data.append(stock_data)
    data['created_at'] = pd.to_datetime(data['created_at'])
    data = data.set_index('created_at')
    return data

In [10]:
# Test using a small search
start_date = dt.date(year=2021, month=3, day=17)
end_date = dt.date(year=2021, month=3, day=28)
print(start_date)
print(end_date)
test_DLPN_df = generate_df(['DLPN'], start_date)

2021-03-17
2021-03-28


Rate limit reached. Sleeping for: 698


In [11]:
test_DLPN_df.index.min()

Timestamp('2021-03-21 22:59:54+0000', tz='UTC')

In [12]:
test_DLPN_df.index.max()

Timestamp('2021-03-30 01:31:56+0000', tz='UTC')

In [15]:
test_DLPN_df.shape

(9881, 320)

In [17]:
test_DLPN_df.to_pickle('DLPN_2021_03_21_to_2021_03_30.pkl')

In [18]:
test_read = pd.read_pickle('DLPN_2021_03_21_to_2021_03_30.pkl')

In [19]:
test_read.shape

(9881, 320)

In [20]:
test_read['stock'].head()

created_at
2021-03-30 01:31:56+00:00    DLPN
2021-03-30 01:28:14+00:00    DLPN
2021-03-30 01:28:04+00:00    DLPN
2021-03-30 01:24:23+00:00    DLPN
2021-03-30 01:22:40+00:00    DLPN
Name: stock, dtype: object

# Testing updates to Tweepy pull - 3/30

In [1]:
import twitter_data

In [2]:
test_df = twitter_data.generate_df(['DPLN', 'CAN', 'MARA'], save_results=True)

Fetching data for DPLN from 2021-03-22 to 2021-03-30 ...
Tweets found: 130
Fetching data for CAN from 2021-03-22 to 2021-03-30 ...
Tweets found: 1437


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Fetching data for MARA from 2021-03-22 to 2021-03-30 ...
Tweets found: 3992


In [3]:
test_df = twitter_data.read_all_twitter_data()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
test_df.shape

(5133, 321)

In [7]:
test_df.created_at.min()

'Fri Mar 26 00:03:50 +0000 2021'

In [6]:
test_df.index.min()

Timestamp('2021-03-23 12:14:20+0000', tz='UTC')

## Trying searchtweets

In [65]:
import searchtweets

In [66]:
premium_search_args = searchtweets.load_credentials("keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [67]:
rule = searchtweets.gen_rule_payload("$DLPN", results_per_call=100) # testing with a sandbox account
print(rule)

{"query": "$DLPN", "maxResults": 100}


In [68]:
tweets = searchtweets.collect_results(rule, max_results=100, result_stream_args=premium_search_args) # change this if you need to

HTTP Error code: 400: {"errors":[{"code":86,"message":"This method requires a GET or HEAD."}]}
Request payload: {'query': '$DLPN', 'maxResults': 100}


HTTPError: 